# Оригинальная модель

In [ ]:
# %pip install datasets

In [1]:
from transformers import AutoImageProcessor, SwinForImageClassification
import torch
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset

In [2]:
# @title Среда
import torch
from transformers import AutoImageProcessor
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

device: cuda


In [3]:
# @title Данные
from datasets import load_dataset
dataset = load_dataset("mvkvc/artifact-100k") # subset of ArtiFact labled with "ai" and "real" tags

dataset = dataset.with_format(type="torch", device=device)

image_processor = AutoImageProcessor.from_pretrained("umm-maybe/AI-image-detector")

batch_size = 40

train_loader = torch.utils.data.DataLoader(dataset=dataset["train"], batch_size=batch_size,  num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=dataset["test"], batch_size=batch_size, num_workers=2)

C:\Users\bes-s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [4]:
image_processor.do_normalize = True
image_processor.size

{'height': 230, 'width': 230}

In [6]:
# @title Модель

PATH = 'checkpoints/1-1/'
model = SwinForImageClassification.from_pretrained(PATH, local_files_only=True)
model.classifier = torch.nn.Sequential(
    # torch.nn.Linear(in_features=768, out_features=2, bias=True),
    torch.nn.Sigmoid(),
    torch.nn.Linear(in_features=1024, out_features=2, bias=True),
    torch.nn.Sigmoid()
    )
model.to(device)
print("Модель загружена")
print(model)

Some weights of the model checkpoint at checkpoints/1-1/ were not used when initializing SwinForImageClassification: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing SwinForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SwinForImageClassification were not initialized from the model checkpoint at checkpoints/1-1/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Модель загружена
SwinForImageClassification(
  (swin): SwinModel(
    (embeddings): SwinEmbeddings(
      (patch_embeddings): SwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): SwinEncoder(
      (layers): ModuleList(
        (0): SwinStage(
          (blocks): ModuleList(
            (0-1): 2 x SwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): SwinAttention(
                (self): SwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
                (o

In [7]:
from transformers import AutoImageProcessor, SwinForImageClassification
import torch
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset

# dataset = load_dataset("huggingface/cats-image")

# image_processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
# model = SwinForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

# Finetune

In [8]:
# @title EarlyStopping
class EarlyStopping:
    def __init__(self, value=0.7, patience=5):
        self.value = value
        self.patience = patience
        self.wait = 0
        self.stopped_epoch = 0
        self.best = -float('inf')
        self._improved_this_step = False

    def improved_this_step(self) -> bool:
        return self._improved_this_step

    def _condition(self, value) -> bool:
        return False

    def __call__(self, epoch, value) -> bool:
        self._improved_this_step = False
        if self._condition(value):
            self.best = value
            self.wait = 0
            self._improved_this_step = True
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                print(f"Epoch {epoch}: early stopping")
                return True
        return False

class EarlyStoppingByAccuracy(EarlyStopping):
    def _condition(self, value) -> bool:
        return value > self.best

In [ ]:
# for batch_ind, data in enumerate(test_loader):
#      inputs, targets = data['image'], data['label']
#      targets = torch.nn.functional.one_hot(targets, num_classes = 2).to(torch.float)
#      print (targets)
#      break

In [16]:
# @title Training loop { display-mode: "form" }

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# loss_func = torch.nn.L1Loss()
loss_func = torch.nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.8)
optimizer = optim.Adam(model.parameters(), lr=0.08)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5, verbose=True)

import torch
import datetime
from torch.utils.tensorboard import SummaryWriter



def calculate_epoch(writer: SummaryWriter, epoch, is_train=False):
  data_loader = train_loader if is_train else test_loader
  if is_train:
    model.train(True)
  else:
    model.eval()

  total = 0
  correct = 0
  avg_loss = 0
  batches = 0

  for batch_ind, data in enumerate(data_loader):
    inputs, targets = data['image'], data['label']
    inputs = inputs.permute(0,3,1,2)
    inputs, targets = inputs.to(torch.float32).to(device), targets.to(torch.long).to(device)
    # targets = targets.unsqueeze(1)

    optimizer.zero_grad()
    output = model(inputs).logits
    total += len(output)
    correct += (output.argmax() == targets).float().sum()

    targets = torch.nn.functional.one_hot(targets, num_classes = 2).to(torch.float)
    loss = loss_func(output, targets)
    
    if is_train:
      loss.backward()
      optimizer.step()
    avg_loss += loss.item()
    batches += 1

  avg_loss = avg_loss / batches
  accuracy = correct/total

  stage = "train" if is_train else "test"
  writer.add_scalar(f"loss/{stage}", avg_loss, epoch)
  writer.add_scalar(f"accuracy/{stage}", accuracy, epoch * total)
  if stage == "train":
    current_lr = optimizer.param_groups[0]['lr']
    writer.add_scalar('Learning Rate', current_lr, epoch)
  return {"acc": accuracy, "avg_loss" : avg_loss}


def train_loop():
  time_now = datetime.datetime.now()
  layout = {
      "Model statistics" : {
      "loss": ["Multiline", ["loss/train", "loss/test"]],
      "accuracy": ["Multiline", ["accuracy/train", "accuracy/test"]],
      "learning_rate": ["Scalar", "Learning Rate"]
      }
  }
  writer = SummaryWriter(log_dir=time_now.strftime("runs/SWIN/run %d-%m %H_%M_%S"))
  writer.add_custom_scalars(layout)

  early_stop = EarlyStoppingByAccuracy()
  epoch_limit = 1000
  epoch = 0
  is_training = True
  while is_training and epoch < epoch_limit:
    print(f"epoch #{epoch+1}/{epoch_limit}...", end = "")
    time_start = datetime.datetime.now()
    calculate_epoch(writer, epoch, is_train=True)

    report = None
    with torch.no_grad():
      report = calculate_epoch(writer, epoch, is_train=False)
      scheduler.step(report["avg_loss"])
    if early_stop(epoch, report["acc"]):
      is_training = False


    if early_stop.improved_this_step():
      model.save_pretrained(f"./checkpoints/{epoch}", safe_serialization=False) # saves as SafeTensors

      
    time_end = datetime.datetime.now()
    deltatime = time_end - time_start
    print(f"done in {deltatime.seconds/60:4f} minutes")
    writer.flush()
    epoch += 1

  writer.close()
  print('done training.')

train_loop()

epoch #1/1000...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128}


done in 13.283333 minutes
epoch #2/1000...done in 12.983333 minutes
epoch #3/1000...done in 12.983333 minutes
epoch #4/1000...done in 12.983333 minutes
epoch #5/1000...done in 12.983333 minutes
epoch #6/1000...Epoch 5: early stopping
done in 12.983333 minutes
done training.


In [ ]:
model.save_pretrained("./checkpoints", safe_serialization=True) # saves as SafeTensors